In [36]:
import numpy as np
import pandas as pd
from keras.preprocessing import image,load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os



In [37]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [38]:
filenames=os.listdir("extracted_images")

categories=[]
for f_name in filenames:
    category=f_name.split('.')[0]
    if category=='dog':
        categories.append(1)
    else:
        categories.append(0)

df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})
df

,filename,category
0,cat.0.jpg,0
1,cat.1.jpg,0
2,cat.10.jpg,0
3,cat.100.jpg,0
4,cat.1000.jpg,0
...,...,...
24995,dog.9995.jpg,1
24996,dog.9996.jpg,1
24997,dog.9997.jpg,1
24998,dog.9998.jpg,1


**Creating The Model by using Neural Net**

In [39]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization

model=Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 batch_normalization_4 (Bat  (None, 126, 126, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 63, 63, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_5 (Bat  (None, 61, 61, 64)       

In [41]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',patience=2,verbose=1,factor=0.5,min_lr=0.00001)
callbacks=[earlystop,learning_rate_reduction]

In [42]:
df['category']=df['category'].replace({0:'cat',1:'dog'})
train_df, validate_df =train_test_split(df,test_size=0.20,random_state=42)
train_df   =train_df.reset_index(drop=True)
validate_df=validate_df.reset_index(drop=True)
total_train= train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [43]:
train_datagen = ImageDataGenerator(rotation_range=15,rescale=1./255,shear_range=0.1,zoom_range=0.2,
                                   horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_df,"./extracted_images/",x_col='filename',y_col='category',
                                                    target_size = Image_Size,class_mode='categorical',batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df, "./extracted_images", x_col='filename',y_col='category',
                                                              target_size=Image_Size,class_mode='categorical',batch_size=batch_size)
test_datagen = ImageDataGenerator(rotation_range=15,rescale=1./255,shear_range=0.1,zoom_range=0.2,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df, "./extracted_images/",x_col='filename',y_col='category',target_size=Image_Size,
                                                 class_mode='categorical',batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 20000 validated image filenames belonging to 2 classes.


**Model Training**

In [44]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

C:\Users\Nikhil\AppData\Local\Temp\ipykernel_21420\2528005629.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10


1333/1333 [==============================] - 553s 413ms/step - loss: 0.7476 - accuracy: 0.6338 - val_loss: 0.5841 - val_accuracy: 0.7171 - lr: 0.0010
Epoch 2/10
1333/1333 [==============================] - 534s 401ms/step - loss: 0.5565 - accuracy: 0.7246 - val_loss: 0.5192 - val_accuracy: 0.7506 - lr: 0.0010
Epoch 3/10
1333/1333 [==============================] - 392s 294ms/step - loss: 0.4978 - accuracy: 0.7659 - val_loss: 0.6040 - val_accuracy: 0.7580 - lr: 0.0010
Epoch 4/10
1333/1333 [==============================] - 325s 244ms/step - loss: 0.4682 - accuracy: 0.7817 - val_loss: 0.4025 - val_accuracy: 0.8134 - lr: 0.0010
Epoch 5/10
1333/1333 [==============================] - 342s 257ms/step - loss: 0.4461 - accuracy: 0.7946 - val_loss: 0.3886 - val_accuracy: 0.8284 - lr: 0.0010
Epoch 6/10
1333/1333 [==============================] - 344s 258ms/step - loss: 0.4238 - accuracy: 0.8074 - val_loss: 0.4926 - val_accuracy: 0.7788 - lr: 0.0010
Epoch 7/10
1333/1333 [==========

In [57]:
model.save("model_catsVSdogs_10epoch.h5")

c:\Users\Nikhil\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [58]:
test_filenames = os.listdir("./test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [60]:
predictions = model.predict(test_generator, steps=test_generator.n // test_generator.batch_size + 1)

1334/1334 [==============================] - 174s 131ms/step


In [101]:
test_df['category'] = np.argmax(predictions, axis=-1)

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [105]:
from keras.preprocessing.image import load_img
import matplotlib.pyplot as plt

sample_test = test_df.head(18)
sample_test.head()

plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = str(row['filename'])
    category = row['category']

    # Skip if filename is 'nan'
    if filename == 'nan':
        continue

    img_path = "./test1/" + filename

    try:
        img = load_img(img_path,filename, target_size=Image_Size)
        plt.subplot(6, 3, index+1)
        plt.imshow(img)
        plt.xlabel(filename + '(' + "{}".format(category) + ')' )
    except FileNotFoundError:
        print(f"File not found: {img_path}")

plt.tight_layout()
plt.show()


<Figure size 1200x2400 with 0 Axes>

In [110]:
from PIL import Image
import numpy as np

results = {0: 'cat', 1: 'dog'}

# Assuming you have a list of image filenames
image_filenames = ["x.0.jpg", "x.1.jpg", "y.1.jpg"]

for filename in image_filenames:
    img_path = "./test1/" + filename

    try:
        im = Image.open(img_path)
        im = im.resize(Image_Size)
        im = np.expand_dims(im, axis=0)
        im = np.array(im)
        im = im / 255
        pred_probabilities = model.predict([im])[0]
        pred_class = np.argmax(pred_probabilities)
        print(f"Prediction for {filename}: {results[pred_class]}")
    except FileNotFoundError:
        print(f"File not found: {img_path}")


1/1 [==============================] - 0s 47ms/step
Prediction for x.0.jpg: cat
1/1 [==============================] - 0s 32ms/step
Prediction for x.1.jpg: dog
1/1 [==============================] - 0s 40ms/step
Prediction for y.1.jpg: cat


In [111]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))